## Implement Wiedmann work on PxI SUT

In [ ]:
# reviewing paper
# TIMs = m are 'scope 3' calculated via
# m = f*inv(I-A) = f*L where 
# f is direct intensity factors (DIMS)
# that is, the total impact F divided by the industry output
# F could be a matrix of several impacts, or just one impact by industry
# A is the 'technology coefficient' matrix which is the product of the 
# T table (transaction table) and the inverse of industry output
# ie 'normalized input-output'



In [ ]:
# example data
import pandas as pd

sut = pd.read_excel('data/wiedmann.xlsx',
                    sheet_name='su_table',
                    index_col=0)
num_industries = 6
num_products = 10

In [ ]:
# extract the various pieces as arrays
# make them linalg objects and compute derivative objects
import numpy as np

x = sut['output'].drop(index='Emissions').to_numpy()
y = sut['demand'].drop(index='Emissions').to_numpy()
f = sut.drop(columns=['output','demand']).loc['Emissions'].to_numpy()
sut = sut.drop(index='Emissions',columns=['output','demand']).to_numpy()

x_hat = np.diag(x) # correct
tech_coef = np.dot(sut,np.linalg.inv(x_hat)) # correct
dims = np.dot(f,np.linalg.inv(x_hat)) # correct
leontief = np.linalg.inv(np.eye(x.shape[0])-tech_coef) # correct
tims = np.dot(dims,leontief) # correct


In [ ]:
# decomposing by industry
tims_by_industry = np.dot(np.diag(dims),leontief) # correct
# matrix of TIMs decomposed by industry
# all supply chain paths that start with industry i (and its impact) and end up
# with product p sum up to the total share of industry i in the TIM of product p
    # see series-expansion derivation for explanation
# the top-right block of this table is most useful for footprinting as
# products are in demand, not industries (top left block)


In [ ]:
# The carbon map is a decomposition of the total CF of the final demand for products 
# np.round(np.dot(m_industry,np.diag(y))) # correct

In [ ]:
# the contribution of specific products to another product's impact is not
# quantified in this analysis, but is most useful for LCI of specific goods
disaggregated_tims = np.diag(dims) + np.dot(np.diag(tims),tech_coef)



array([[0.18018018, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.64877466, 0.64877466, 0.05724482, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.32967033, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.51831781, 0.55461229,
        0.21451985, 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.32258065, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.01292612,
        0.32998187, 0.52997088, 0.10599418, 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.09090909, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.23959112, 0.29948889, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.55555556,
        0.        , 0.        , 

In [ ]:
# wait. Are you just computing the dot product in the transpose?
# product_id = 
# print(product_id)
a = np.atleast_2d(tech_coef[:num_industries,:])
m_ip = disaggregated_tims[:,:num_industries]
# this is actually the result we want in entirety,
# with zeros in the LH block
m_p = np.dot(m_ip,a)
print(np.round(m_p[:,num_industries:],3)) # product block
print(m_p[:,:num_industries]) # industry block

In [ ]:
# # product 2: TIM = 0.57
# def s_matrix(product_id):
#     # pick the product column from the tech coef 
#     a = tech_coef[:num_industries,num_industries+product_id]
#     # and just the respective column from the TIMs
#     m_ip = disaggregated_tims[:,:num_industries]

#     # then it's the broadcast multiplication 
#     s = a * m_ip
#     return s

# prod 0 and 1 are the same actually
product_id = 2
m_out = np.zeros((num_industries+num_products,num_products))
for product_id in range(10):
    a = tech_coef[:num_industries,num_industries+product_id]
    m_ip = disaggregated_tims[:,:num_industries]
    # then it's the broadcast multiplication 
    s_p = a * m_ip
    # sum across the columns (i.e. compute row sums)
    m_p_p = np.sum(s_p,axis=1)
    m_out[:,product_id] = m_p_p
# this is column number p
m_out # correct

array([[0.18018018, 0.18018018, 0.01589825, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.30058177, 0.32162959, 0.1244039 ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.00786782, 0.2008521 ,
        0.32258065, 0.06451613, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.07272727, 0.09090909, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.55555556, 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 1.        ],
       [0.02337927, 0.02337927, 0.02156386, 0.02154711, 0.02544538,
        0.02790429, 0.02209512, 0.02064283, 0.01982367, 0.00763264],
       [0.        , 0.        , 0.0162508

In [ ]:
# check equality
np.all(np.isclose(m_out,m_p[:,num_industries:],np.zeros_like(m_out)))

True